In [3]:
import h5py
import tensorflow as tf

In [9]:
class train_generator:
    def __call__(self):
      with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(int(len(hf['X'])*0.8)): # both hf1 and hf2 have same length
                yield hf['X'][i], hf['y'][i]
                yield hf2['X'][i], hf2['y'][i]                

In [6]:
with h5py.File('data/electron.hdf5', 'r') as hf:
  test_ds_start=int(len(hf['X'])*0.8)+1
test_ds_start

199201

In [10]:
class test_generator:
    def __call__(self):
      with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(int(len(hf['X'])*0.2)): # both hf1 and hf2 have same length
                yield hf['X'][test_ds_start+i], hf['y'][test_ds_start+i]
                yield hf2['X'][test_ds_start+i], hf2['y'][test_ds_start+i]  

In [11]:
train_ds = tf.data.Dataset.from_generator(
  train_generator(),
  output_signature=(
    tf.TensorSpec(shape=(32, 32, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
  )
)
test_ds = tf.data.Dataset.from_generator(
  test_generator(),
  output_signature=(
    tf.TensorSpec(shape=(32, 32, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
  )
)

In [15]:
iter_train_ds=iter(train_ds)
def input_fn():
    feature, label = next(iter_train_ds)
    return {"x": feature}, label


In [17]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.FlatMapDataset

In [20]:
batched_train_ds=train_ds.batch(32)
batched_test_ds=test_ds.batch(32)

In [21]:
iter_batched_train_ds=iter(batched_train_ds)
def input_fn():
    feature, label = next(iter_batched_train_ds)
    return {"x": feature}, label

In [30]:
feature, label = next(iter_train_ds)
tf..reduce_max(feature)

SyntaxError: invalid syntax (1543761546.py, line 2)

In [ ]:
train_ds, test_ds=tf.keras.utils.split_dataset(batched_ds, left_size=0.8)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape=(209, 209, 3)),
  
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
  
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),


    tf.keras.layers.Flatten(),
   
    tf.keras.layers.Dense(512, activation='relu'),
   
  
    tf.keras.layers.Dense(1, activation='sigmoid')
])